In [1]:
!pip show ultralytics

Name: ultralytics
Version: 8.3.19
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: https://ultralytics.com
Author: 
Author-email: Glenn Jocher <glenn.jocher@ultralytics.com>, Jing Qiu <jing.qiu@ultralytics.com>
License: AGPL-3.0
Location: /home/simon/miniconda3/envs/simba_310/lib/python3.10/site-packages
Requires: matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, torch, torchvision, tqdm, ultralytics-thop
Required-by: 


In [2]:
import numpy as np
import os
from simba.third_party_label_appenders.converters import geometries_to_yolo
from simba.utils.read_write import find_files_of_filetypes_in_directory, read_df, get_fn_ext, find_video_of_file
from simba.mixins.geometry_mixin import GeometryMixin
from simba.bounding_box_tools.yolo.model import fit_yolo, inference_yolo
from simba.bounding_box_tools.yolo.visualize import YOLOVisualizer
from simba.third_party_label_appenders.converters import simba_rois_to_yolo, split_yolo_train_test_val


In [3]:
TRAIN_DATA_DIR = r'/mnt/c/troubleshooting/mitra/project_folder/csv/yolo_train' # DIRECTORY CONTAINING SIMBA CSV FILES USED FOR TRAINING YOLO MODEL
VIDEO_DIR = r'/mnt/c/troubleshooting/mitra/project_folder/videos' # DIRECTORY HOLDING THE VIDEOS REPRESENTING THE FILES IN THE ``TRAIN_DATA_DIR``
YOLO_SAVE_DIR = r"/mnt/c/troubleshooting/yolo_animal" # DIRECTORY WHERE TO SAVE THE THE YOLO-FORMATTED DATA
BODY_PARTS_HULL = ['Nose_x', 'Nose_y', 'Tail_base_x', 'Tail_base_y', 'Left_side_x', 'Left_side_y', 'Right_side_x', 'Right_side_y'] # THE FIELD NAMES REPRESENTING THE BODY-PARTS BELONGING TO THE OUTHER EDGES OF THE ANIMAL HULL
SAMPLE_COUNT = 400 # MAXIMUM NUMBER OF YOLO LABELS THAT SHOULD BE CREATED FROM EACH VIDEO
VERBOSE = False # SET TO TRUE TO FOLLOW PROGRESS

In [5]:
# WE CREATE RECTANGULAR POLYGON SHAPES FROM THE POSE-ESTIMATED DATA
file_paths = find_files_of_filetypes_in_directory(directory=TRAIN_DATA_DIR, extensions=['.csv'])
video_polygons = {}
for file_path in file_paths:
    animal_data = read_df(file_path=file_path, file_type='csv', usecols=BODY_PARTS_HULL).values.reshape(-1, 4, 2).astype(np.int32)
    animal_polygons = GeometryMixin().multiframe_bodyparts_to_polygon(data=animal_data, verbose=VERBOSE)
    animal_polygons = GeometryMixin().multiframe_minimum_rotated_rectangle(shapes=animal_polygons, verbose=VERBOSE)
    video_polygons[get_fn_ext(file_path)[1]] = GeometryMixin().geometries_to_exterior_keypoints(geometries=animal_polygons)

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

In [6]:
#WE TRANSFORM THE RECTANGULAR POLYGONS INTO YOLO LABELS AND ASSOCIATED IMAGES
for video_name, polygons in video_polygons.items():
    polygons = {0: polygons}
    video_path = find_video_of_file(video_dir=VIDEO_DIR, filename=video_name)
    geometries_to_yolo(geometries=polygons, video_path=video_path, save_dir=YOLO_SAVE_DIR, sample=SAMPLE_COUNT, verbose=VERBOSE, obb=True)

In [7]:
# WE SPLIT THE YOLO LABELS CREATED IN THE PRIOR CELL INTO TRAINING, TEST, AND VALIDATION
YOLO_TRAINING_DIR = r"/mnt/c/troubleshooting/yolo_data_split_animal" # DIRECTORY WHERE WE SHOULD STORE THE SLIP DATA.
split_yolo_train_test_val(data_dir=YOLO_SAVE_DIR, save_dir=YOLO_TRAINING_DIR, verbose=VERBOSE)

In [8]:
#NEXT, WE TRAIN A YOLO MODEL BASED ON THE SPLIT DATA
INITIAL_WEIGHTS_PATH = r'/mnt/c/Users/sroni/Downloads/yolov8n.pt' #SOME INITIAL WEIGHTS TO START WITH, THEY CAN BE DOWNLOADED AT https://huggingface.co/Ultralytics
MODEL_SAVE_DIR = "/mnt/c/troubleshooting/yolo_mdl" #DIRECTORY WHERE TO SAVE THE TRAINED MODEL AND PERFORMANCE STATISTICS
EPOCHS = 25 # HOW MANY EPOCHS TO TRAIN THE MODEL FOR
BATCH = 16 # THE NUMBER OF OBSERVATIONS IN EACH ITERATION

fit_yolo(initial_weights=INITIAL_WEIGHTS_PATH, model_yaml=os.path.join(YOLO_TRAINING_DIR, 'map.yaml'), save_path=MODEL_SAVE_DIR, epochs=EPOCHS, batch=BATCH, plots=True)

New https://pypi.org/project/ultralytics/8.3.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.19 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine/trainer: task=detect, mode=train, model=/mnt/c/Users/sroni/Downloads/yolov8n.pt, data=/mnt/c/troubleshooting/yolo_data_split_animal/map.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/mnt/c/troubleshooting/yolo_mdl, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augmen

train: Scanning /mnt/c/troubleshooting/yolo_data_split_animal/labels/train... 1953 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1953/1953 [00:06<00:00, 302.80it/s]

train: WARNING ⚠️ /mnt/c/troubleshooting/yolo_data_split_animal/images/train/FRR_gq_Saline_0626_24647.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0093]


train: New cache created: /mnt/c/troubleshooting/yolo_data_split_animal/labels/train.cache


val: Scanning /mnt/c/troubleshooting/yolo_data_split_animal/labels/val... 278 images, 0 backgrounds, 0 corrupt: 100%|██████████| 278/278 [00:01<00:00, 196.36it/s]


...

      25/25      2.06G     0.8625     0.7458      1.212         16        640: 100%|██████████| 122/122 [00:08<00:00, 13.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:00<00:00, 10.98it/s]

                   all        278        278       0.85      0.899      0.921      0.723



25 epochs completed in 0.072 hours.
Optimizer stripped from /mnt/c/troubleshooting/yolo_mdl/train/weights/last.pt, 6.2MB
Optimizer stripped from /mnt/c/troubleshooting/yolo_mdl/train/weights/best.pt, 6.2MB

Validating /mnt/c/troubleshooting/yolo_mdl/train/weights/best.pt...
Ultralytics 8.3.19 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  7.52it/s]


                   all        278        278       0.85      0.895      0.921      0.723
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /mnt/c/troubleshooting/yolo_mdl/train


In [ ]:
#NEXT, WE USE THE TRAINED MODEL TO FIND THE ROIS IN A NEW VIDEO
INFERENCE_RESULTS = "/mnt/c/troubleshooting/yolo_results" #DIRECTORY WHERE TO STORE THE RESULTS IN CSV FORMAT
VIDEO_PATH = r'/mnt/c/troubleshooting/mitra/project_folder/videos/501_MA142_Gi_Saline_0513.mp4' #PATH TO VIDEO TO ANALYZE
inference_yolo(weights_path=os.path.join(MODEL_SAVE_DIR, 'train', 'weights', 'best.pt'), video_path=VIDEO_PATH, verbose=VERBOSE, save_dir=INFERENCE_RESULTS, gpu=True, batch_size=BATCH, interpolate=True)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.19 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/mnt/c/troubleshooting/yolo_mdl/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.35...
ONNX: export success ✅ 0.7s, saved as '/mnt/c/troubleshooting/yolo_mdl/train/weights/best.onnx' (11.7 MB)

TensorRT: starting export with TensorRT 10.5.0...
[11/16/2024-11:49:10] [TRT] [I] [MemUsageChange] Init CUDA: CPU +1, GPU +0, now: CPU 2063, GPU 1583 (MiB)
[11/16/2024-11:49:25] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +2238, GPU +426, now: CPU 4457, GPU 2009 (MiB)
[11/16/2024-11:49:25] [TRT] [I] ----------------------------------------------------------------
[11/16/

In [ ]:
#FINALLY, WE VISUALIZE THE RESULTS TO CHECK THAT THE PREDICTIONS ARE ACCURATE
DATA_PATH = r"/mnt/c/troubleshooting/yolo_results/501_MA142_Gi_CNO_0514_clipped.csv" #PATH TO ONE OF THE CSV FILES CREATED IN THE PRIOR STEP.
VIDEO_PATH = r'/mnt/c/troubleshooting/mitra/project_folder/videos/clipped/501_MA142_Gi_CNO_0514_clipped.mp4' # PATH TO THE VIDEO REPRESENTING THE ``DATA_PATH`` FILE.
SAVE_DIR = r"/mnt/c/troubleshooting/yolo_videos" # DIRECTORY WHERE TO SAVE THE YOLO VIDEO

yolo_visualizer = YOLOVisualizer(data_path=DATA_PATH, video_path=VIDEO_PATH, save_dir=SAVE_DIR, palette='Accent', thickness=20, core_cnt=-1, verbose=False)
yolo_visualizer.run()

In [ ]:
#EXPECTED RESULTS

video_url = 'https://raw.githubusercontent.com/sgoldenlab/simba/master/docs/_static/img/YOLOVisualizer.webm'
HTML(f''' <video width="600" height="600" controls> <source src="{video_url}" type="video/mp4"> </video>
''')